<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#functions" data-toc-modified-id="functions-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>functions</a></span></li></ul></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#exploring-&quot;SCIENCE_DATA&quot;" data-toc-modified-id="exploring-&quot;SCIENCE_DATA&quot;-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>exploring "SCIENCE_DATA"</a></span><ul class="toc-item"><li><span><a href="#right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly." data-toc-modified-id="right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly.-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>right, i forgot there's a readme explaining everything quite clearly.</a></span></li><li><span><a href="#so-how-do-i-interpret-the-320x36?" data-toc-modified-id="so-how-do-i-interpret-the-320x36?-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>so how do i interpret the 320x36?</a></span><ul class="toc-item"><li><span><a href="#before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries" data-toc-modified-id="before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries</a></span></li></ul></li></ul></li><li><span><a href="#Data-format-is-based-off-of-instrument-format" data-toc-modified-id="Data-format-is-based-off-of-instrument-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data format is based off of instrument format</a></span></li><li><span><a href="#filename-considerations" data-toc-modified-id="filename-considerations-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>filename considerations</a></span></li><li><span><a href="#right,-i-think-i've-got-the-format" data-toc-modified-id="right,-i-think-i've-got-the-format-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>right, i think i've got the format</a></span><ul class="toc-item"><li><span><a href="#Matrix-matching" data-toc-modified-id="Matrix-matching-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Matrix-matching</a></span></li></ul></li><li><span><a href="#speed-tests" data-toc-modified-id="speed-tests-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>speed tests</a></span></li><li><span><a href="#append-time:-81s-/-.h5" data-toc-modified-id="append-time:-81s-/-.h5-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>append time: 81s / .h5</a></span></li><li><span><a href="#list-zip-time:-78s-/-.h5" data-toc-modified-id="list-zip-time:-78s-/-.h5-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>list zip time: 78s / .h5</a></span></li><li><span><a href="#TIME-issue:" data-toc-modified-id="TIME-issue:-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>TIME issue:</a></span><ul class="toc-item"><li><span><a href="#matrix-to-DataFrame-directly" data-toc-modified-id="matrix-to-DataFrame-directly-1.8.1"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>matrix to DataFrame directly</a></span></li><li><span><a href="#nice.-i-can-just-flatten-the-arrays-and-dataframe-it" data-toc-modified-id="nice.-i-can-just-flatten-the-arrays-and-dataframe-it-1.8.2"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>nice. i can just flatten the arrays and dataframe it</a></span></li></ul></li><li><span><a href="#matrix-test:-much-faster,-but-warning" data-toc-modified-id="matrix-test:-much-faster,-but-warning-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>matrix test: much faster, but warning</a></span></li><li><span><a href="#good-to-remember:-work-with-matrices-when-you-can,-it's-much-faster-than-iterating-i,j" data-toc-modified-id="good-to-remember:-work-with-matrices-when-you-can,-it's-much-faster-than-iterating-i,j-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>good to remember: work with matrices when you can, it's much faster than iterating i,j</a></span><ul class="toc-item"><li><span><a href="#now-build-the-process_all_orbits()" data-toc-modified-id="now-build-the-process_all_orbits()-1.10.1"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>now build the process_all_orbits()</a></span></li></ul></li></ul></li><li><span><a href="#hexbin" data-toc-modified-id="hexbin-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>hexbin</a></span></li></ul></div>

### imports

In [16]:
import h5py
import os
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import time

### functions

In [2]:
def peek_dataset(ds): # only works on 2d array datasets
    for i in range(ds.shape[0]):
        for k in range(ds.shape[1]):
            print(ds[i][k])
        print('\n')
        
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'sat_alt':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df
    

def process_orbit_old(df, h5): # pandas DF, string. adds h5's observations to dataframe
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...')
            obs = {} # 11,520 max
            obs['lat'] = geo['Latitude'][i][k]
            obs['long'] = geo['Longitude'][i][k]
            obs['sat_lat'] = geo['SpacecraftLatitude'][i] # 1d so just use 1st loop
            obs['sat_long'] = geo['SpacecraftLongitude'][i]
            obs['time'] = geo['TimeUTC'][i]
            obs['sza'] = geo['SolarZenithAngle'][i]
            obs['pbl'] = sci['ColumnAmountSO2_PBL'][i][k]
            obs['anom'] = sci['Flag_SAA'][i][k]
            obs['cloud'] = sci['RadiativeCloudFraction'][i][k]
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            df = df.append(obs, ignore_index=True)
            c+=1
    # end loops
    print(f'dataframe length: {len(df)}')
    return df

# EDA

In [3]:
filename = "many_outs.h5"

okay, found the issue
- for downloading multiple .h5 files, nasa reccomends:
    - cat <url.txt> | tr -d '\r' | xargs -n 1 curl -LJO -n -c ~/.urs_cookies -b ~/.urs_cookies
    - but i was getting some access denied + use --output recommendations. didn't work
        - so i tried >> piping it to a file. i could have named it "data.h5", maybe? can you store h5s in other h5s?
        - i named it 'data.iso' on someone's SO solution
        - this went poorly. the data downloaded, but when trying to open it, it seemed super corrupted, even with specific encoding that allowed it to open
    - i went back to NASA's downloading instructions and tested the curl cmd with a smaller subset of .h5 files, which worked (i think i had some typos in the cmdline, or i was formatting incorrectly) so i applied it to the original 2012-2020 46gb of data. currently downloading all the .h5s to my home directory, so i'll just have to move them en masse and loop-open them into a dataframe in the project folder (in 5 hours when it downloads).

In [4]:
# import os
# for filename in os.listdir(os.getcwd()):
#    with open(os.path.join(os.cwd(), filename), 'r') as f: # open in readonly mode
#       # do your stuff

In [5]:
lst = []
for filename in os.listdir(os.getcwd()):
    if filename.endswith('.h5'):
        with h5py.File(filename, 'r') as f:
            print('keys: %s' %f.keys())
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])

keys: <KeysViewHDF5 ['ANCILLARY_DATA', 'GEOLOCATION_DATA', 'SCIENCE_DATA', 'nTimes', 'nWavel2', 'nWavel3', 'nXtrack']>


In [17]:
import h5py
f = h5py.File('one.h5', 'r')

In [7]:
f.keys()

<KeysViewHDF5 ['ANCILLARY_DATA', 'GEOLOCATION_DATA', 'SCIENCE_DATA', 'nTimes', 'nWavel2', 'nWavel3', 'nXtrack']>

In [8]:
for k in f.keys():
    print(k, type(f[k]), '\n') # remember to use the key on the dict obj to actually get the type of value

ANCILLARY_DATA <class 'h5py._hl.group.Group'> 

GEOLOCATION_DATA <class 'h5py._hl.group.Group'> 

SCIENCE_DATA <class 'h5py._hl.group.Group'> 

nTimes <class 'h5py._hl.dataset.Dataset'> 

nWavel2 <class 'h5py._hl.dataset.Dataset'> 

nWavel3 <class 'h5py._hl.dataset.Dataset'> 

nXtrack <class 'h5py._hl.dataset.Dataset'> 



- so the root h5py object is a dictionary... of sorts.
    - we've got Groups, which are like folders, and Datasets, which are like arrays (but different)

In [9]:
f['GEOLOCATION_DATA'].keys() # looks like location of Suomi satellite at moment of reading
# we're gonna want Latitude and Longitude to identify 'where' the reading is.
# maybe TimeUTC as well, that could save a lot of re-coding work for broad regional targeting

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [10]:
f['ANCILLARY_DATA'].keys()

<KeysViewHDF5 ['CloudPressure', 'TerrainPressure']>

In [11]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

from the NASA dataset description:

The goal of this dataset is to create and archive a Level 2 Sulfur Dioxide (SO2) Earth Science Data Record (ESDR) from backscatter Ultraviolet (BUV) measurements from the Ozone Mapping and Profiler Suite (OMPS) Nadir Mapper (NM) flown on the NASA/NOAA Suomi National Polar-orbiting Partnership (S-NPP) satellite since 2011. We apply the NASA Goddard Space Flight Center (GSFC) principal component analysis (PCA) satellite trace gas retrieval algorithm, to obtain the best measurement-based ESDR of volcanic and anthropogenic SO2 loadings and emissions, which is fully consistent with our SO2 ESDR (OMSO2) from the Ozone Monitoring Instrument (OMI) flown on the NASA Earth Observing System (EOS) Aura satellite since 2004. 

The SO2 ESDR released here at the Goddard Earth Science (GES) Data and Information Data Center (DISC), OMPS_NPP_NMSO2_PCA_L2, is a stand-alone dataset, but will also be part of the NASA's Making Earth System Data Records for Use in Research Environments (MEaSUREs) program. It covers the entire period of the S-NPP/OMPS mission since launch in October 2011, and forward data processing is ongoing. OMPS_NPP_NMSO2_PCA_L2 is a Level 2 orbital swath product, which will be used to study SO2 pollutants emitted from large point sources such as coal-fired power plants and smelters as well as to monitor volcanic SO2 emissions.

Sulfur Dioxide (SO2) is a short-lived gas primarily produced by volcanoes, power plants, refineries, metal smelting and burning of fossil fuels. Where SO2 remains near the Earth's surface, it is toxic, causes acid rain, and degrades air quality. Where SO2 is lofted into the free troposphere, it forms aerosols that can alter cloud reflectivity and precipitation. In the stratosphere, volcanic SO2 forms sulfate aerosols that can result in climate change.

- translation:
    - here's SO2 levels by latitude, date & many other variables
        - derived from applying PCA algos to Ozone mapping data, which was measured from backscatter ultraviolet readings
                - basically, NASA looks at backscattered UV off of the atmosphere - off of ozone, but can effectively derive SO2 from the ozone reading using PCA
        - seems convoluted, but it's consistent with their last SO2 reading system from 2004, and it's probably the best estimate the world's got right now (unless anyone would like to step up and accurately measure trace gas concentrations globally every day. seems bloody hard in my opinion)

## exploring "SCIENCE_DATA"

In [12]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

In [13]:
f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

<HDF5 dataset "ColumnAmountSO2_PBL": shape (320, 36), type "<f4">

In [14]:
pbl = f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

In [15]:
geo.keys()

NameError: name 'geo' is not defined

In [ ]:
geo['TimeUTC'][310]

In [ ]:
# peek_dataset(pbl) # a 320x36 matrix. now to figure out why there's 11520 readings in this dataset, which is one .h5 file


In [ ]:
for k in f['SCIENCE_DATA'].keys(): # are they all 320x36? probably the SO2 at least
#     print(k, type(k))
    if 'SO2' in k: # k is a string
        print(k, f['SCIENCE_DATA'][k].shape) # yes, the SO2 readings are matched in size.
# probably different instruments or estimation methods. perhaps we could aggregate them later on, but they're all going in the dataframe for now

### right, i forgot there's a readme explaining everything quite clearly.
- units
    - 5 estimates (principal components), related to various geophysical processes (ozone absorption, rotational Raman scattering) and instrument measurement details (wavelength shift, dark current)
    - PCs used in SO2 spectral fitting to reduce interferences
        - so we've got five estimates of total "SO2 Vertical Column Density" in Dobson Units (1DU = 2.69 * 10^16 molecules/cm^2)
            - note the "cm^2". these are flat "slices" of a column? no
            - each of the 5 estimates corresponds to a different assumed SO2 cloud height, or Center of Mass Altitude (CMA)
            - all 5 VCD values represent the estimated total SO2 burden within entire atmospheric column:
                - should not be interpreted as partial column amounts within different layers/parts of atmosphere
                - they want users to select 1 or interpolate between two estimates that are most representative of conditions for a particular case of interest
- so each of the 5 estimates measures with different heights/sensitivities, and therefore are 'tuned' to measure different... types of SO2 emission.
    - anthropogenic (coal factories etc) SO2 emissions will look different, and be measured differently from volcanic eruptions
    - that sort of thing

- PBL: Planetary Boundary Layer
    - assumes that SO2 is predominantly in lowest 1km of atmosphere
    - use for studies on near-surface pollution
        - for each OMPS orbit:
            - process 36 rows (cross-track positions) one at a time
            - use PCA to extract PCs for spectral range 310.5-340nm from sun-normalized BUV radiance spectra
            - PCs are ranked in descending order according to spectral variance that they explain
    - due to jacobian simplication, they recommend only using snow/ice free pixels with small radiative cloud fraction
- the other 4 column amounts seem to be volcanic
### so how do i interpret the 320x36?
- wait, they're talking about pixels
    - 320x36 is the "resolution" of the measurement
    
#### before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries

- it's an "orbital-track" product about the SNPP satellite.
    - polar sun-synchronous orbit
## Data format is based off of instrument format
- OMPS-NM has 110° FoV, cross-track swath ~= 2800km
    - global coverage,14-15 orbits/day
    - "nominal spatial resolution" is 50km x 50km at nadir in nominal observation mode
        - there's a higher res mode recorded once a week, but that's not in this dataset
- for each OMPS orbit, process 36 rows (cross-track positions) use PCA to extract PCs for spectral range from BUV
    - so, 320 PCs per position while crossing orbital track?
        - what does the lat/long data look like? that might solve this question


In [ ]:
lat = f['GEOLOCATION_DATA']['Latitude']
long = f['GEOLOCATION_DATA']['Longitude']
print(lat.shape, long.shape)

- so in one .h5's 36x320:
    - began at -29, -152: about 1/3 of the way from NZ to Ecuador
    - ended at 67, 16: northern middle Norway
        - but those aren't the extreme bounds for lat or long.
            - without plotting it out, my intuition says it's an oscillating path

In [ ]:
# peek_dataset(lat)
# peek_dataset(long)

- now the 50km spatial resolution makes sense.
    - we've got 11520 'pixels' or 'granules' (are granules the .h5?)
    - each pixel has an SO2 PBL reading, as well as a lat/long
        - so the corresponding [x],[y] position in the matrix will match a geolocation to its PBL
- warnings from the readme:
    - all pixels:
        - with large solar zenith angle (SZA>70)
        - near edge of swath (rows 1-2, 35-36)
        - affected by South Atlantic Anomaly (flag_SAA=1)
            - should be excluded
    - PBL SO2:
        - only use:
            - snow/ice free pixels
            - with RadiativeCloudFraction (<0.3)
            
    

## filename considerations
- each .h5 file contains:
    - date/time at start of orbit
    - orbit number
        - so each .h5 is an orbit. question answered
- what about time? that should help me figure out "unique positions"

In [ ]:
geo = f['GEOLOCATION_DATA']
time = geo['TimeUTC']
time.shape # ah

In [ ]:
geo.keys()

In [ ]:
lat = geo['Latitude']
lat.shape

In [ ]:
lat[0][5]

- so we've got 320 times per orbit, but 320x36 pixels and 320x36 lat/long coords
- that means at 320 times throughout the day, the satellite gathers 36 pixels, and each one has a unique geolocation...?
    - the 36 is "cross-track positions"
        - is it a vertical "36 slices of column underneath spacecraft"?
            - no
            - if that were the case, 
                - there wouldn't be a separate lat/long for spacecraft as well
                    - the "spacecraft lat/long" is (320) while "lat/long" is (320,36)
                    - lines up with 320 timestamps
    - how to interpret "cross-track positions"
        - 36 positions that cross the track (orbital track path over earth's surface)
        - so for each 320 position+timestamp along track, there's 36 "positions" that cross the track
        - i'm assuming it does a left/right sweep below in its 110° fov while continuing "forwards"
            - this makes the most sense. spacecraft geocoords are 1d, geocoords are 2d, so the 2nd dimension from line to plane has to be added somehow
            - i could be stupendously wrong, but let's worry about that later

## right, i think i've got the format
- each .h5 file is an orbit, with 320 track positions (geocoords+timestamp)
    - each track position has 36 cross-track positions, so we get 320x36 matrix for readings+GPS
    - each track/crosstrack combo is a 50km x 50km space of earth (i think...)
        - the lat/long is decimals, but that can be converted to degrees if needed
- for each .h5 file, we've got 11,520 PBL readings, but not all of them will be usable
    - ice/snow, large solar zenith angle, south atlantic magnetic interference
- and we've got something like... 40,000 or 50,000 h5s, maybe more
### Matrix-matching
- lat[5][6], long[5][6] and PBT[5][6] will give us the location and reading for the 5th position, 6th cross position
    - so we can just write functions to neatly iterate through each h5's file tree and add each 'observation' to a big dataframe
    - it's going to be big


## speed tests
- append or concat? 11,520 matrix elements, 36,000ish orbits

In [ ]:


start = time.time()
print("hello")
end = time.time()
print(end - start)

In [ ]:
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df

def process_orbit_app(df, h5): # pandas DF, string. adds h5's observations to dataframe
    start = time.time()
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...')
            obs = {} # 11,520 max
            obs['lat'] = geo['Latitude'][i][k]
            obs['long'] = geo['Longitude'][i][k]
            obs['sat_lat'] = geo['SpacecraftLatitude'][i] # 1d so just use 1st loop
            obs['sat_long'] = geo['SpacecraftLongitude'][i]
            obs['time'] = geo['TimeUTC'][i]
            obs['sza'] = geo['SolarZenithAngle'][i]
            obs['pbl'] = sci['ColumnAmountSO2_PBL'][i][k]
            obs['anom'] = sci['Flag_SAA'][i][k]
            obs['cloud'] = sci['RadiativeCloudFraction'][i][k]
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            df = df.append(obs, ignore_index=True)
            c+=1
    # end loops
    print(f'dataframe length: {len(df)}')
    end = time.time()
    print(f'TIME .append()  :  {end - start}')
    return df

def process_orbit_list(df, h5): # pandas DF, string. adds h5's observations to dataframe
#     start = time.time()
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    lat, long, sat_lat, sat_long, time, sza, pbl, anom, cloud = ([] for i in range(9)) # create 9 empty lists
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...') # add to each list
            lat.append(geo['Latitude'][i][k])
            long.append(geo['Longitude'][i][k])
            sat_lat.append(geo['SpacecraftLatitude'][i]) # 1d so just use 1st loop
            sat_long.append(geo['SpacecraftLongitude'][i])
            time.append(geo['TimeUTC'][i])
            sza.append(geo['SolarZenithAngle'][i])
            pbl.append(sci['ColumnAmountSO2_PBL'][i][k])
            anom.append(sci['Flag_SAA'][i][k])
            cloud.append(sci['RadiativeCloudFraction'][i][k])
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            c+=1
    # end loops
    df = pd.DataFrame(list(zip(lat, long, sat_lat, sat_long, time, sza, pbl, anom, cloud)),
                     columns=["lat", "long", "sat_lat", "sat_long", "time", "sza", "pbl", "anom", "cloud"])
    print(f'dataframe length: {len(df)}')
#     end = time.time()
#     print(f'TIME .append()  :  {end - start}')
    return df

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)


In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

In [ ]:
pd.concat([df3, df4], axis=1)

In [ ]:
# list of strings 
lst = ['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks'] 
  
# list of int 
lst2 = [11, 22, 33, 44, 55, 66, 77] 
lst3 = ['a','b','c','d','e','f','g']
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(lst, lst2, lst3)), 
               columns =['Name', 'val', 'chr']) 

In [ ]:
df

In [ ]:

filename = 'one.h5'
df = create_base_df()
start = time.time()
df = process_orbit_app(df, filename)
end = time.time()
print(f'TIME .append()  :  {end - start}')

## append time: 81s / .h5
## list zip time: 78s / .h5
- closer than i thought. i thought i read .concat() was faster? but then again i'm not using concat in the end - that combines dataframes

In [ ]:
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
os.getcwd()

In [ ]:
os.listdir(os.getcwd())

## TIME issue:
- 80 seconds per h5 for 38k h5s gives us 844 hours of runtime for the function.
    - that is absolutely uncool
- is it an unavoidable issue with the amount of data in terms of functional calls?
    - i'm dealing with h5.Datasets, which are basically arrays
        - i have a feeling there's a faster way to array->dataframe than nested loop appending
    

In [ ]:
len(files)

In [ ]:
sci.keys()

### matrix to DataFrame directly

In [ ]:
mat = sci['ColumnAmountSO2_PBL']
mat.shape

In [ ]:
pd.DataFrame()

In [ ]:
m = pd.DataFrame(mat) # 320 times a day, 36 cross-track positions are measured
m # could we just compress the 2d matrix to 1d, keeping track every 36?

- i have 9 matrices, each 320x36 for 11520 observations
    - or rather, ~4 are just 320 ( but i could extend them into 320x36 )
    - how can i make a dataframe shaped (11520, 9)
    - pretty sure pd.concat() will work here
- i think flattening it out to 1d could simplify things greatly
    - is there any reason to keep 2d shape? no...?
        - from a data structure standpoint, the 2d shape tells us:
            - "which observations belong to the same track-position?"
        - so if we flatten all matrices equally (and for the 1d, just multiply by 36)
            - then the structure should be solved / positions preserved?
        - then if i have 9 (11520) matrices, i basically have a dataframe

In [ ]:
matrix = mat.value # convert to nparray
matrix

In [ ]:
matrix.shape

In [ ]:
pbl = matrix.ravel() # looking at the 1st and last values, it seems like order is preserved (why wouldn't it be?)
pbl

In [ ]:
pbl.shape

In [ ]:
sci.keys()

In [ ]:
clouds = sci['RadiativeCloudFraction'].value.ravel()

In [ ]:
clouds.shape

In [ ]:
clouds_lst = list(clouds)
pbl_lst = list(pbl)
df = pd.DataFrame(list(zip(clouds_lst, pbl_lst)), columns=['clouds', 'pbl'])
df

In [ ]:
df2 = df.copy() # deep


In [ ]:
print(df.shape, df2.shape)

In [ ]:
d3 = pd.concat([df,df2])
d3

### nice. i can just flatten the arrays and dataframe it
- this problem was tough until i let go of the notion that i need the 320x36 shape
    - when you're stuck, reconsidering your assumptions/accepted 'requirements' can help
- so the new process_h5_matrix() function:
    - open h5
    - assign datasets to vars
    - if 2d: flatten to 1d
    - if 1d: multiply length by 36
    - list & zip into DF

In [ ]:
df = create_base_df()
start = time.time()
df = process_orbit(df, filename)
end = time.time()
print(f'time:  {end-start}')

In [ ]:
df.shape

## matrix test: much faster, but warning
time:  0.01427316665649414
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


## good to remember: work with matrices when you can, it's much faster than iterating i,j 

In [ ]:
filename

### now build the process_all_orbits()

In [ ]:
!pip install tqdm

In [ ]:
st = f'{1}s'
st

In [ ]:
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'sat_alt':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df
    

def process_orbit(h5): # pandas DF, filename string. adds h5's observations to dataframe
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    # hdf.Datasets -> np.array -> list
    lat = list(geo['Latitude'].value.ravel())
    long = list(geo['Longitude'].value.ravel()) # is there a less verbose way to do this?
    sat_lat = list(geo['SpacecraftLatitude'].value.ravel())*36 # extend the 1d arrays
    sat_long = list(geo['SpacecraftLongitude'].value.ravel())*36 # don't forget to ravel like i did
    sat_alt = list(geo['SpacecraftAltitude'].value.ravel())*36
    time = list(geo['TimeUTC'].value.ravel())*36 # 36 measurements per position means one "time" value for every consecutive 36 measurements
    sza = list(geo['SolarZenithAngle'].value.ravel())
    pbl = list(sci['ColumnAmountSO2_PBL'].value.ravel())
    anom = list(sci['Flag_SAA'].value.ravel())
    # combine lists into df
    new = pd.DataFrame(list(zip(lat, long, sat_lat, sat_long, sat_alt, time, sza, pbl, anom)),
                       columns=["lat", "long", "sat_lat", "sat_long", 'sat_alt', "time", "sza", "pbl", "anom"])
    # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
#     df = pd.concat([df, new]) # merge 11,520 into existing df

#     print(len(df))
    return new # returns new df

In [ ]:
from tqdm import tqdm
# def process_all_h5(df): # loads all h5s into a dataframe. will save locally every ~500 files
#     mypath = os.getcwd() + "/data/" # get all files in data folder
#     files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#     for f in tqdm(files): # h5py is kind of picky with absolute filenames & root folder files
#         path = mypath+f
#         df = process_orbit(df, path) # ah, that did the trick
        # enumerate instead of tqdm to allow saving every 100 .h5s
    # more stable in case of crash. could also write this to upload to mysql & keep track of processed/unprocessed files
def process_all_h5(df): # loads all h5s into a dataframe. will save locally every ~500 files
    mypath = os.getcwd() + "/data/" # get all files in data folder
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for c,f in enumerate(files): # h5py is kind of picky with absolute filenames & root folder files
        print(c)
        path = mypath+f
        new = process_orbit(path)
        df = pd.concat([df, new])
#         if c/100 == 0:
#             st = f'increment_{c}'
#             df.to_csv(st+'.csv')
        print(f'LENGTH:  {len(df)}')
        df.to_csv('increm.csv')
    return df

df = create_base_df()
start = time.time()
df = process_all_h5(df)
df.to_csv('all_data.csv')
end = time.time()
print(f'time:   {end-start}')

In [ ]:
2/100


In [ ]:
d = pd.read_csv('increm.csv')
len(d)

In [ ]:
df.to_csv('data.csv')

In [ ]:
lst = []
for filename in os.listdir(os.getcwd()):
    if filename.endswith('.h5'):
        with h5py.File(filename, 'r') as f:
            print('keys: %s' %f.keys())
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])

In [18]:
geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
sci = f['SCIENCE_DATA']

In [19]:
sci.keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

In [36]:
for k in sci.keys():
    print(k, type(sci[k][0][0]), sci[k][0][0])

CloudFraction <class 'numpy.float32'> 0.21159439
ColumnAmountO3 <class 'numpy.float32'> 281.70758
ColumnAmountSO2_PBL <class 'numpy.float32'> -0.40295467
ColumnAmountSO2_STL <class 'numpy.float32'> -0.060004227
ColumnAmountSO2_TRL <class 'numpy.float32'> -0.17981637
ColumnAmountSO2_TRM <class 'numpy.float32'> -0.089290515
ColumnAmountSO2_TRU <class 'numpy.float32'> -0.06284826
FittingWindow_STL <class 'numpy.ndarray'> [313. 340.]
FittingWindow_TRL <class 'numpy.ndarray'> [313. 340.]
FittingWindow_TRM <class 'numpy.ndarray'> [313. 340.]
FittingWindow_TRU <class 'numpy.ndarray'> [313. 340.]
Flag_SAA <class 'numpy.int32'> 0
Flag_SO2 <class 'numpy.int32'> 0
RadiativeCloudFraction <class 'numpy.float32'> 0.29958948
Reflectivity331 <class 'numpy.float32'> 0.28990737
SLER <class 'numpy.ndarray'> [0.29474375 0.30017498 0.3100103 ]
UVAerosolIndex <class 'numpy.float32'> 0.54779524
Wavelengths_SLER <class 'numpy.ndarray'> [342.66742 353.91214 366.8322 ]
dNdR <class 'numpy.ndarray'> [-35.882015 -

In [31]:
for k in geo.keys():
    try:
        print(k, type(geo[k][0][0]), geo[k][0][0])
    except:
        print('1d', k, type(geo[k][0]), geo[k][0])

Latitude <class 'numpy.float32'> -29.434868
Longitude <class 'numpy.float32'> -152.67432
SolarAzimuthAngle <class 'numpy.float32'> -14.087847
SolarZenithAngle <class 'numpy.float32'> 53.389477
1d SpacecraftAltitude <class 'numpy.float32'> 836490.5
1d SpacecraftLatitude <class 'numpy.float32'> -25.943134
1d SpacecraftLongitude <class 'numpy.float32'> -139.42757
1d TimeTAI93 <class 'numpy.float64'> 613090702.888068
TimeUTC <class 'int'> 50
ViewingAzimuthAngle <class 'numpy.float32'> 76.658226
ViewingZenithAngle <class 'numpy.float32'> 66.20959


In [47]:
time = geo['TimeUTC']

In [33]:
time[0]

b'2012-06-05T22:58:15.888068Z'

In [34]:
type(time[0])

numpy.bytes_

In [48]:
tim = time[0]

In [49]:
d = {'col1': [tim, tim], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

,col1,col2
0,b'2012-06-05T22:58:15.888068Z',3
1,b'2012-06-05T22:58:15.888068Z',4


In [54]:
df.dtypes

col1    object
col2     int64
dtype: object

In [60]:
# df['col1'] = df['col1'].apply(lambda t: t.tostring())
df['col1'] = df['col1'].astype(str)

In [61]:
df['col1'][0]

"b'2012-06-05T22:58:15.888068Z'"

In [62]:
type(df['col1'][0])

str

In [38]:
st = time[0].tostring() # bytes to str
st

b'2012-06-05T22:58:15.888068Z'

In [45]:
st = "b'2012-06-05T22:58:15.888068Z'" # change NASA format to mysql DATETIME
timest = st[2:12]+' '+st[13:21]
timest

'2012-06-05 22:58:15'

# hexbin
hexbin code was somewhere in this notebook? let's find it again for GIS testing elsewhere